In [1]:
import copy
from konlpy.tag import Okt
import pykospacing
import kss
from nltk.tokenize import TreebankWordTokenizer
from wordcloud import WordCloud
#불용어를 제거하기 위해 만든 stopwords 파일을 읽어옵니다.
with open('data/stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
#stopword 파일을 작성할 때 썼던 줄바꿈 문자들을 전부 제거합니다.
stopwords = [x.replace('\n','') for x in stopwords]
okt = Okt()
tokenizer = TreebankWordTokenizer()
def word_preprocess_korean(text):
    my_text = copy.copy(text)
    #먼저 줄바꿈 문자들을 전부 제거합니다.
    my_text = my_text.replace('\n', '')
    spacer = pykospacing.Spacing()
    #이 후 띄어쓰기가 잘못된 경우 전부 다 수정합니다.
    my_text = spacer(my_text) 
    #본문 내용에 대해 각 문장으로 분리합니다.
    sents = kss.split_sentences(my_text)
    result = []
    for sent in sents:
        #각 문장에 대해서 품사를 기준으로 단어들을 분리합니다.
        morphs = okt.pos(sent)
        for word, pos in morphs:
            #명사이면서 단어의 길이가 1보다 큰 단어들만 리스트에 포함시킵니다.
            if pos in ['Noun'] and len(word) >= 2:
                result.append(word)
    return result

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
df_lst = ['환경','환경문제','환경해결','환경원인','wordcloud_파생_정책관련','wordcloud_파생_오염관련']
for i in df_lst:
    file_name = f"{i}.csv"
    df_dust = pd.read_csv(file_name, encoding = 'utf-8')
    df_dust = df_dust.dropna().reset_index(drop = True)
    #데이터 프레임을 불러와 중복 제거를 다시 하고 아까 만들었던 함수를 적용합니다.
    df_dust['lda_preprocessed_document'] = df_dust['content'].apply(lambda x: word_preprocess_korean(x))
    dust = ''
    #word cloud를 위해 전처리한 모든 명사들을 하나의 문자열로 합칩니다.
    for lst in df_dust['lda_preprocessed_document']:
        dust = " ".join(lst)
    cloud = WordCloud(font_path = 'C:\\Users\\user\\anaconda3\\envs\\text\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\malgun.ttf', 
                  background_color = 'white', 
                  width=800, height=800)
    my_cloud2 = cloud.generate_from_text(dust)

    arr2 = my_cloud2.to_array()

    fig = plt.figure(figsize=(10, 10))
    plt.title(f"{i} Word Cloud")
    plt.imshow(arr2)
    plt.axis('off')
    plt.show()
    #word cloud를 그린 후 'word cloud'라는 폴더 안에 그림을 저장했습니다.
    file_name = f"{i}.png"
    save_dir = os.path.join(os.getcwd(), 'wordcloud')
    file_path = os.path.join(save_dir, file_name)
    fig.savefig(file_path, format = 'png')
    plt.close()